In [1]:
from torch import nn

# Transformer 모델 정의
class Transformer(nn.Module):

  def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512,
               nhead=8, num_encoder_layers=6, num_decoder_layers=6,
               dim_feedforward=2048, dropout=0.1):
    super(Transformer, self).__init__()

    # 인코더와 디코더에 사용할 임베딩 레이어 정의
    self.src_embedding = nn.Embedding(src_vocab_size, d_model)
    self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
    # src : source, tgr : target

    # 포지셔널 인코딩 레이어 정의
    self.positional_encoding = PositionalEncoding(d_model)

    # 트랜스포머 인코더 레이어 정의
    encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
                                               nhead=nhead,
                                               dim_feedforward=dim_feedforward,
                                               dropout=dropout)
    
    encoder_norm = nn.LayerNorm(d_model)
    
    self.encoder = nn.TransformerEncoder(encoder_layer,
                                         num_layers=num_encoder_layers,
                                         norm=encoder_norm)
    
    # 트랜스포머 디코더 레이어 정의
    decoder_layer = nn.TransformerDecoderLayer(d_model=d_model,
                                               nhead=nhead,
                                               dim_feedforward=dim_feedforward,
                                               dropout=dropout)
    decoder_norm = nn.LayerNorm(d_model)
    
    self.decoder = nn.TransformerDecoder(decoder_layer,
                                         num_layers=num_decoder_layers,
                                         norm=decoder_norm)

    # 출력층 정의
    self.out = nn.Linear(d_model, tgt_vocab_size)




  def forward(self, src, tgt):
        
    # 입력과 출력 문장에 임베딩 적용
    src_emb = self.src_embedding(src) * math.sqrt(self.d_model) 
    tgt_emb = self.tgt_embedding(tgt) * math.sqrt(self.d_model) 


    # 입력과 출력 문장에 포지셔널 인코딩 적용
    src_pos_emb = self.positional_encoding(src_emb)
    tgt_pos_emb = self.positional_encoding(tgt_emb)

    
    # 마스크 생성
    src_key_padding_mask = (src == PAD_IDX).transpose(0, 1) 
    tgt_key_padding_mask = (tgt == PAD_IDX).transpose(0, 1) 
    memory_key_padding_mask = (src == PAD_IDX).transpose(0, 1) 
    tgt_mask = generate_square_subsequent_mask(tgt.size(0)).to(device)
    # generate_square_subsequent_mask() 
    # 특정 sequence 길이를 가지는 마스크 생성 함수 

    # 인코더에 소스 문장을 입력하여 메모리 생성
    memory = self.encoder(src_pos_emb.transpose(0, 1),
                          mask=None,
                          src_key_padding_mask=src_key_padding_mask)

    # 디코더에 메모리와 타겟 문장을 입력하여 출력 생성
    output = self.decoder(tgt_pos_emb.transpose(0 ,1),
                          memory.transpose(0 ,1),
                          tgt_mask=tgt_mask,
                          memory_mask=None,
                          tgt_key_padding_mask=tgt_key_padding_mask,
                          memory_key_padding_mask=memory_key_padding_mask)

    # 출력을 선형 변환하여 최종 예측 생성
    output_logits=self.out(output.transpose(0 ,1))

    return output_logits